In [ ]:
pip install openai==0.28


In [ ]:
pip install coverage


# GENERATE TEST CASES

In [1]:
import ast  
import openai
import os
import sys
import system_prompt
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")
model_output = ''
script_path = ''
processed_content = ''
coverage_percentage = 0

def clean_model_output(output: str) -> str:
    """
    Clean the model's output by removing unwanted artifacts like ```python and extra backticks.
    Ensure proper formatting of the test cases.
    """
    # Remove ```python, ```, or similar markers
    cleaned_output = output.replace("```python", "").replace("```", "").strip()

    # Ensure the output contains assert statements; otherwise, return an empty string
    if "assert" not in cleaned_output:
        print("No valid 'assert' statements found in the output.")
        return ""

    # Split lines and reformat to ensure proper indentation for test cases
    return "\n    ".join(line.strip() for line in cleaned_output.splitlines())



# A function to generate the initial prompt to call the model
def initalPromptGenerator(function_to_test):
    initial_prompt = f"""Generate test cases for the following function:
Generate test cases for
```python
{function_to_test}

# test case
def test():
    assert  
"""
    return initial_prompt


# a function to generate fewshot prompt to call the model
def fewshotPromptGenerator(function_to_test,fewshot_content):
    fewshot_prompt = f"""
{fewshot_content}

{function_to_test}

#</code>

#<test>

Strictly generate test cases for above function only and strictly write it in test() function . No string apart from test cases must be given.
def test():
    assert 


"""
    return fewshot_prompt


# Call prompt on OpenAI model   
def call_prompt_on_model(
    prompt: str,
    stop_w: str,
    max_tokens: int,
    code_model: str = "gpt-3.5-turbo",  # Use a chat-based model
    temperature: float = 0.8,
    reruns_if_empty: int = 2,
    reruns_no_assert: int = 2,
) -> str:
    # Call the chat-completions endpoint
    explanation_response = openai.ChatCompletion.create(
        model=code_model,
        messages=[
            {"role": "system", "content": "You are a test case writer."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=temperature,
        stop=stop_w,
    )

    # Extract response content
    explanation_completion = explanation_response["choices"][0]["message"]["content"]

    # Retry logic for empty or missing 'assert'
    if reruns_if_empty > 0 and not explanation_completion.strip():
        print("empty output")
        return call_prompt_on_model(
            prompt=prompt,
            stop_w=stop_w,
            code_model=code_model,
            max_tokens=max_tokens,
            temperature=temperature,
            reruns_if_empty=reruns_if_empty - 1,
            reruns_no_assert=reruns_no_assert,
        )
    if reruns_no_assert > 0 and "assert" not in explanation_completion:
        print("no assert in output")
        return call_prompt_on_model(
            prompt=prompt,
            stop_w=stop_w,
            code_model=code_model,
            max_tokens=max_tokens,
            temperature=temperature,
            reruns_if_empty=reruns_if_empty,
            reruns_no_assert=reruns_no_assert - 1,
        )

    return explanation_completion

# Call the model on a specific prompt and return the output
def call_LLMs(initial_prompt, stop_w, max_tokens):
    uncleaned_output = call_prompt_on_model(initial_prompt, stop_w, max_tokens)
    output = clean_model_output(uncleaned_output)
    # print("Initial Prompt:")
    # print(initial_prompt)
    # print("############")
    # print("Model Output:")
    # print(output)
    # print("############")

    return output

# Read the script from the file, generate the prompt, call the model, and save the output
def generate_test_oracle_from_initial_file(input_path,fewshot_path):
    with open(input_path) as input_file:
        function_to_test = input_file.read()
    with open(fewshot_path) as input_file:
        fewshot_content = input_file.read()
    initial_prompt = fewshotPromptGenerator(function_to_test,fewshot_content)
    global model_output
    model_output = call_LLMs(initial_prompt, "#</test>", 1000)

    # print("Generated Test Cases:")
    # print(model_output)
    # print("############")

    # Combine function and generated test cases
    file_content = f"{function_to_test}\n\ndef test():\n{model_output}"

    return file_content

import os

# Function to generate the test oracle initial file
def generate_test_oracle_initial(script, fewshot):
    name1 = script.split('\\')
    name = name1[-1]
    # print(name)
    fewshot_path = fewshot
    file_content = generate_test_oracle_from_initial_file(script, fewshot_path)
    # print("File content before processing:\n", file_content)
    
    # Process file_content to remove duplicate 'def test()'
    lines = file_content.splitlines()
    seen_test_def = False
    processed_lines = []
    
    for line in lines:
        # Check if the line starts with 'def test():'
        if line.strip() == 'def test():':
            if not seen_test_def:
                seen_test_def = True
                processed_lines.append(line)
        else:
            processed_lines.append(line)
    
    # Join the processed lines back
    global processed_content
    processed_content = '\n'.join(processed_lines)
    print("Test Cases:\n", processed_content)
    
    # Write to output.py
    output_path = os.path.join(os.path.dirname(script), name[:-3]+"_output.py")
    with open(output_path, "w") as output_file:
        output_file.write(processed_content)


    print(f"Test cases written to: {output_path}")



# Main function to execute the script
def main():
    global script_path
    script_path = 'C:\\Data\\MutantGenerator\\script_1.py'
    fewshot_path = 'C:\\Data\\MutantGenerator\\fewshots.py'
    generate_test_oracle_initial(script_path,fewshot_path)

if __name__ == '__main__':
    main()


Test Cases:
 def any_int(x, y, z):
  
    
    if isinstance(x,int) and isinstance(y,int) and isinstance(z,int):
        if (x+y==z) or (x+z==y) or (y+z==x):
            return True
        return False
    return False



def test():
    # Test cases for any_int function
    assert any_int(2, 3, 5) == True  # 2 + 3 = 5
    assert any_int(-2, 3, 1) == True  # -2 + 3 = 1
    assert any_int(0, 0, 0) == True  # 0 + 0 = 0
    assert any_int(-2, 5, -3) == False  # -2 + 5 != -3
    assert any_int(2, 2, 3) == False  # Not possible to form a valid equation
    
    test()
Test cases written to: C:\Data\MutantGenerator\script_1_output.py


In [2]:
def clean_model_output(output: str) -> str:
    """
    Clean the model's output by removing unwanted artifacts like ```python and extra backticks.
    Ensure proper formatting of the test cases.
    """
    # Remove ```python, ```, or similar markers
    cleaned_output = output.replace("```python", "").replace("```", "").strip()

    # Ensure the output contains assert statements; otherwise, return an empty string
    if "assert" not in cleaned_output:
        print("No valid 'assert' statements found in the output.")
        return ""

    # Split lines and reformat to ensure proper indentation for test cases
    return "\n    ".join(line.strip() for line in cleaned_output.splitlines())


def call_prompt_on_model(
    prompt: str,
    stop_w: str,
    max_tokens: int,
    code_model: str = "gpt-3.5-turbo",  # Use a chat-based model
    temperature: float = 0.8,
    reruns_if_empty: int = 2,
    reruns_no_assert: int = 2,
) -> str:
    # Call the chat-completions endpoint
    explanation_response = openai.ChatCompletion.create(
        model=code_model,
        messages=[
            {"role": "system", "content": "You are a test case writer."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=temperature,
        stop=stop_w,
    )

    # Extract response content
    explanation_completion = explanation_response["choices"][0]["message"]["content"]

    # Retry logic for empty or missing 'assert'
    if reruns_if_empty > 0 and not explanation_completion.strip():
        print("empty output")
        return call_prompt_on_model(
            prompt=prompt,
            stop_w=stop_w,
            code_model=code_model,
            max_tokens=max_tokens,
            temperature=temperature,
            reruns_if_empty=reruns_if_empty - 1,
            reruns_no_assert=reruns_no_assert,
        )
    if reruns_no_assert > 0 and "assert" not in explanation_completion:
        print("no assert in output")
        return call_prompt_on_model(
            prompt=prompt,
            stop_w=stop_w,
            code_model=code_model,
            max_tokens=max_tokens,
            temperature=temperature,
            reruns_if_empty=reruns_if_empty,
            reruns_no_assert=reruns_no_assert - 1,
        )

    return explanation_completion

def updateTestCases(existingFunction):
    
    prompt = f"""
You are senior tester at openai and you need to analyze the given Python function and its existing test cases. Generate **new test cases** that:
1. **Correctness**: Ensure the test cases accurately verify the function’s expected behavior.
2. **Edge Cases**: Include boundary values, such as minimum and maximum possible inputs.
3. **Negative & Zero Handling**: If applicable, test how the function handles negative numbers and zero.
4. **Variety**: Use diverse inputs, including valid and invalid values, to ensure robustness.
5. **Coverage**: Test different execution paths, including special cases like empty inputs, large numbers, and uncommon scenarios.
6. **Maintainability**: Append new test cases at the end of the existing test function while keeping the format consistent.

Maintain the existing test cases and append the new ones at the end of the test function.

Function:
```python
{existingFunction}
```

Generate test cases in such a way that coverage becomes 100%

**Note**: Striclty provide only the test cases

**Output format**
```
def test():
    <testcases>
```

Answer: 
"""

    return prompt


def update_processed_content(processed_lines):
    global processed_content
    processed_content = '\n'.join(processed_lines)


global processed_content
print(processed_content)
print("------------------------------------------------------------------------------------------------------------------------------------------------------")
print("Are you satisfied with above test cases: (Yes/No)")
ans = input()
print(ans)
print("------------------------------------------------------------------------------------------------------------------------------------------------------")
while ans.lower()=='no':
    name1 = script_path.split('\\')
    name = name1[-1]
    script = script_path

    prompt = updateTestCases(processed_content)
    ans = call_prompt_on_model(prompt, None, 2000)
    file_content = clean_model_output(ans)

    # Process file_content to remove duplicate 'def test()'
    lines = file_content.splitlines()
    seen_test_def = False
    processed_lines = []

    for line in lines:
        if line.strip() == 'def test():':
            if not seen_test_def:
                seen_test_def = True
                processed_lines.append(line)
        else:
            processed_lines.append(line)

    update_processed_content(processed_lines)

    # print("File content after processing:\n", processed_content)

    output_path = os.path.join(os.path.dirname(script), name[:-3] + "_output.py")
    # print(output_path)
    with open(script_path) as input_file:
        function_to_test = input_file.read()
    with open(output_path, "w") as output_file:
        output_file.write(f"{function_to_test}\n\n{processed_content}")

    # Save the content in testing_file for coverage check
    with open(output_path) as output_file:
        output_content = output_file.read()
        model_output=output_content
        print(output_content)
        print("------------------------------------------------------------------------------------------------------------------------------------------------------")
        print("Are you satisfied with above test cases: (Yes/No)")
        
    ans=input()
    print(ans)
    print("------------------------------------------------------------------------------------------------------------------------------------------------------")





def any_int(x, y, z):
  
    
    if isinstance(x,int) and isinstance(y,int) and isinstance(z,int):
        if (x+y==z) or (x+z==y) or (y+z==x):
            return True
        return False
    return False



def test():
    # Test cases for any_int function
    assert any_int(2, 3, 5) == True  # 2 + 3 = 5
    assert any_int(-2, 3, 1) == True  # -2 + 3 = 1
    assert any_int(0, 0, 0) == True  # 0 + 0 = 0
    assert any_int(-2, 5, -3) == False  # -2 + 5 != -3
    assert any_int(2, 2, 3) == False  # Not possible to form a valid equation
    
    test()
------------------------------------------------------------------------------------------------------------------------------------------------------
Are you satisfied with above test cases: (Yes/No)
no
------------------------------------------------------------------------------------------------------------------------------------------------------
def any_int(x, y, z):
  
    
    if isinstance(x,int) and isinstance(y,int) and isinst

# GENERATE MUTANTS

In [3]:
import openai

def generate_mutants(
    prompt: str,
    stop_w: str,
    max_tokens: int,
    code_model: str = "gpt-3.5-turbo",
    temperature: float = 0.8
) -> str:
    # Call the chat-completions endpoint
    explanation_response = openai.ChatCompletion.create(
        model=code_model,
        messages=[
            {"role": "system", "content": "You are an experienced mutant generator."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=temperature,
        stop=stop_w,
    )

    # Extract response content
    return explanation_response["choices"][0]["message"]["content"]

def getMutantPrompt(function_to_test):
    prompt = f'''
    "I want you to act as a mutation testing assistant. Your task is to generate at least **10 different mutants** for the following code. 
    Apply multiple mutation techniques including:
    
    - **Value Mutations:** Modify constants or parameters by adding, subtracting, or scaling values.
    - **Decision Mutations:** Alter condition expressions (> → <, >= → <=, flip boolean logic, etc.).
    - **Statement Mutations:** Delete or reorder lines, change arithmetic operations, or omit crucial updates (e.g., missing increments in loops).

    **Ensure that each mutant is different and introduces a new possible fault in the original code.** 
    Output the mutants in a structured format, each with a label describing the applied mutation.

    Here is the original code:

    ```python
    {function_to_test}
    ```

    **Generate at least 10 mutants in each request.** If the user is unsatisfied, generate 10 more new ones."
    '''
    return prompt

def newMutantPrompt(previousMutants):
    prompt = f"""
    You are a highly experienced mutant generator with 10 years of expertise in code mutation. Given the previous mutants, generate **at least 10 new ones** while keeping the existing ones unchanged. Apply the following mutation types:

    - **Value Mutations:** Modify constants or parameters (e.g., add, subtract, or scale values).
    - **Decision Mutations:** Alter conditional expressions (> → <, >= → <=, flip boolean logic, etc.).
    - **Statement Mutations:** Delete or reorder lines, change arithmetic operations, or omit crucial updates (e.g., missing increments in loops).
    
    **Do not repeat previous mutants.** Ensure all new mutants introduce fresh variations.

    **Previous Mutants:**
    ```python
    {previousMutants}
    ```

    **Generate 10 more unique mutants in each request.**
    """
    return prompt

# script_1.py (Question path)
input_path = script_path

with open(input_path) as input_file:
    function_to_test = input_file.read()

# Get initial mutant prompt
prompt = getMutantPrompt(function_to_test)
mutants = generate_mutants(prompt=prompt, stop_w=None, max_tokens=2000)
all_mutants = mutants  # Store all mutants

print(mutants)
print("Are you satisfied with the above mutants? (Yes/No)")
isSatisfied = input().strip().lower()
print(isSatisfied)
while isSatisfied == 'no':
    mutantPrompt = newMutantPrompt(all_mutants)
    all_mutants = generate_mutants(prompt=mutantPrompt, stop_w=None, max_tokens=2000)
    all_mutants += "\n" + all_mutants  # Append new mutants
    
    print(all_mutants)
    print("Are you satisfied with the new mutants? (Yes/No)")
    isSatisfied = input().strip().lower()
    print(isSatisfied)

Here are 10 different mutants generated for the original code using various mutation techniques:

1. **Mutant 1 - Value Mutation (Condition Change):**
   ```python
   def any_int(x, y, z):
       if isinstance(x,int) and isinstance(y,int) and isinstance(z,int):
           if (x+y==z) and (x+z==y) and (y+z==x):  # Changed 'or' to 'and'
               return True
           return False
       return False
   ```

2. **Mutant 2 - Decision Mutation:**
   ```python
   def any_int(x, y, z):
       if isinstance(x,int) and isinstance(y,int) and isinstance(z,int):
           if (x+y==z) or (x+z!=y) or (y+z==x):  # Changed '==' to '!='
               return True
           return False
       return False
   ```

3. **Mutant 3 - Statement Mutation:**
   ```python
   def any_int(x, y, z):
       if isinstance(x,int) and isinstance(y,int) and isinstance(z,int):
           if (x+y==z) or (y+z==x):
               return True
           return False
       return False
   ```

4. **Mutant 4 - Value

# CLEANING THE MUTANTS

In [4]:
import re
import os

count = 0
input_data = all_mutants  # Assuming `ans` contains your input string
print(all_mutants)
# Patterns to find code blocks inside triple backticks
pattern_python = r'```python(.*?)```'
# pattern_generic = r'```(.*?)```'

matches = []
matches.extend(re.findall(pattern_python, input_data, re.DOTALL))
# matches.extend(re.findall(pattern_generic, input_data, re.DOTALL))
temp = input_path.split('\\')
folder = temp[-1][:-3] + "_mutant"
directory_path = f'C:\\Data\\MutantGenerator\\{folder}'

# Create directory if it does not exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

for match in matches:
    output_file = f'{directory_path}\\mutant_generated_{count}.py'
    
    with open(output_file, 'w') as outfile:
        outfile.write(match.strip() + "\n")
    
    count += 1

print("Content extracted and saved to", count, "files.")


Certainly! Here are 10 new unique mutants generated based on the previous mutations:

1. **Mutant 11 - Value Mutation (Scaling):**
   ```python
   def any_int(x, y, z):
       if isinstance(x,int) and isinstance(y,int) and isinstance(z,int):
           if (x+y==z) or (x+z==2*y) or (y+z==x):  # Scaled 'z' by 2
               return True
           return False
       return False
   ```

2. **Mutant 12 - Decision Mutation:**
   ```python
   def any_int(x, y, z):
       if isinstance(x,int) and isinstance(y,int) and isinstance(z,int):
           if (x+y==z) or (x+z==y) and (y-z==x):  # Changed 'or' to 'and', added subtraction
               return True
           return False
       return False
   ```

3. **Mutant 13 - Statement Mutation:**
   ```python
   def any_int(x, y, z):
       if isinstance(x,int) and isinstance(y,int) and isinstance(z,int):
           if (x+y==z) and (x+z==y) and (y-z==x):  # Added subtraction operation
               return True
           return False
       

# TESTING THE MUTANT

In [5]:
def get_mutant_prompt(mutant,test_cases):
    content = ''
    with open(mutant,'r') as f:
        content = f.read()
    mutant_not_survived_prompt = f'''
    You are given a mutant that did not survive on the following test cases:

    Mutant: {content}
    Existing Test Cases: {test_cases}

    Your task is to generate a set of new test cases that:
    1. Maintain the original logic of the previous test cases.
    2. Ensure that the new test cases cover any edge cases, boundary conditions, or potential areas the previous tests might have missed.
    3. Check for correctness, robustness, and proper handling of inputs that may not have been fully considered.
    4. Are specifically designed to expose the issue in the mutant's behavior.

    Please take into account the following details when generating the new test cases:
    - The function should handle both typical and edge cases efficiently.
    - Ensure that the test cases include both positive and negative scenarios where appropriate.
    - If possible, create corner cases that might trigger unintended behaviors, such as null inputs, large numbers, or extreme edge values.
    - Return me only the test case with one indentation block.
    - Ensure that you add new test cases in existing test cases
'''
    return mutant_not_survived_prompt

def LLMCall(
    prompt: str,
    stop_w: str,
    max_tokens: int,
    code_model: str = "gpt-3.5-turbo",  # Use a chat-based model
    temperature: float = 0.8,
    reruns_if_empty: int = 2,
    reruns_no_assert: int = 2,
) -> str:
    # Call the chat-completions endpoint
    
    explanation_response = openai.ChatCompletion.create(
        model=code_model,
        messages=[
            {"role": "system", "content": "You are mutants generator."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=temperature,
        stop=stop_w,
    )

    # Extract response content
    explanation_completion = explanation_response["choices"][0]["message"]["content"]


    return explanation_completion

In [6]:
import os
import subprocess
import re

# Folder containing the mutants
mutants_folder = directory_path
# Variable containing test cases
# print(model_output)
print("-------------------------------------------------------------------------------------------------------------------------------------")
print(model_output)
print("-------------------------------------------------------------------------------------------------------------------------------------")

# Regular expression to capture content inside `def test():`, including all indented lines
pattern = r"def test\(\):(\n(?:\s{4}.*\n*)+)"  # Matches all lines that are properly indented under `def test()`

match = re.search(pattern, model_output)

if match:
    test_function = match.group(0)  # Extract the full test function
    test_cases_var = re.findall(r"(\s{4}assert\s+.*)", test_function)  # Extract assert statements with indentation
    test_cases_variable = "\n".join(test_cases_var)  # Store all test cases with indentation

    print("Extracted Test Function:")
    print("----------------------------------------------------------------")
    print(test_function)  # Output the full test function
    print("----------------------------------------------------------------\n")

    print("Extracted Test Cases:")
    print("----------------------------------------------------------------")
    print(test_cases_variable)  # Output only assert statements with indentation
    print("----------------------------------------------------------------")
else:
    print("No test cases found.")

test_cases = f"""
def test():
{test_cases_variable}
    

test()
"""
print(test_cases)
print("-------------------------------------------------------------------------------------------------------------------------------------")

def AddTestCases(new_test_case,old_test_case):
    prompt = f'''
    Add new test_cases below old test cases with proper indentation and give me final output. Strictly only generate new test cases to kill the mutants.
    old test case:
    {old_test_case} without test() function at end.
    new test case:
    {new_test_case}
    put test() function at the end
    '''
    
    ans = LLMCall(prompt,None,1000)
    return ans



def run_tests(mutant_file, test_cases):
    # Read the original content of the mutant file
    with open(mutant_file, 'r') as f:
        original_content = f.read()
    
    # Add the test cases to the mutant file
    with open(mutant_file, 'a') as f:
        f.write('\n' + test_cases)
    
    # Run the mutant file
    result = subprocess.run(['python', mutant_file], capture_output=True, text=True)
    
    # Restore the original content of the mutant file
    with open(mutant_file, 'w') as f:
        f.write(original_content)
    
    return result
testCases =''
def main():
    # Iterate through all mutant files in the folder
    for mutant_file in os.listdir(mutants_folder):
        if mutant_file.endswith('.py'):
            mutant_path = os.path.join(mutants_folder, mutant_file)
            # print(f"Testing mutant: {mutant_file}")
            global test_cases
            result = run_tests(mutant_path, test_cases)
            
            if result.returncode != 0:
                print("Mutant Killed on original test cases")
                # Mutant did not survive
                # print(f"Mutant {mutant_file} did not survive.")
                
            else:
                # Mutant survived
                # print(f"Mutant {mutant_file} survived.")
    
                get_prompt = get_mutant_prompt(mutant_path, result.stderr)
                # print(f"Called LLM for path {mutant_path}")
                ans = LLMCall(get_prompt, None, 1000)
                # print(ans)
                global testCases
                testCases = AddTestCases(ans,test_cases)
                print(testCases)

                # Run the mutant with the new test cases
                result = run_tests(mutant_path, testCases)
                
                if result.returncode == 0:
                    print(f"Mutant {mutant_file} survived with new test cases.")
                else:
                    print(f"Mutant {mutant_file} killed on new test cases.")

                    
    # print(testCases)
    print("Testing completed.")

if __name__ == '__main__':
    main()

-------------------------------------------------------------------------------------------------------------------------------------
def any_int(x, y, z):
  
    
    if isinstance(x,int) and isinstance(y,int) and isinstance(z,int):
        if (x+y==z) or (x+z==y) or (y+z==x):
            return True
        return False
    return False



def test():
    # Test cases for any_int function
    assert any_int(2, 3, 5) == True  # 2 + 3 = 5
    assert any_int(-2, 3, 1) == True  # -2 + 3 = 1
    assert any_int(0, 0, 0) == True  # 0 + 0 = 0
    assert any_int(-2, 5, -3) == False  # -2 + 5 != -3
    assert any_int(2, 2, 3) == False  # Not possible to form a valid equation
    assert any_int(1, 1, 2) == True  # 1 + 1 = 2
    assert any_int(3, 4, 7) == True  # 3 + 4 = 7
    assert any_int(-5, -3, -8) == True  # -5 + (-3) = -8
    assert any_int(10, 10, 20) == True  # 10 + 10 = 20
    assert any_int(100, 200, 301) == False  # 100, 200 and 301 doesn't form a valid equation
    assert any_int(1,

# PUTTING FINAL TEST CASES BACK INTO THE OUTPUT

In [7]:
# print(function_to_test)
# print(testCases)
flag = 0
if testCases:
    # Split the string by lines and remove the first line
    lines = testCases.strip().split('\n')[2:]

    # Join the remaining lines back into a single string
    testCases = '\n'.join(lines)
    ans = f'''
{function_to_test}
def test():
{testCases}

    '''
else:
    flag=1
    ans=model_output
if ans[-1]=='`':
    ans = ans[:-3]
print(ans)

def any_int(x, y, z):
  
    
    if isinstance(x,int) and isinstance(y,int) and isinstance(z,int):
        if (x+y==z) or (x+z==y) or (y+z==x):
            return True
        return False
    return False



def test():
    # Test cases for any_int function
    assert any_int(2, 3, 5) == True  # 2 + 3 = 5
    assert any_int(-2, 3, 1) == True  # -2 + 3 = 1
    assert any_int(0, 0, 0) == True  # 0 + 0 = 0
    assert any_int(-2, 5, -3) == False  # -2 + 5 != -3
    assert any_int(2, 2, 3) == False  # Not possible to form a valid equation
    assert any_int(1, 1, 2) == True  # 1 + 1 = 2
    assert any_int(3, 4, 7) == True  # 3 + 4 = 7
    assert any_int(-5, -3, -8) == True  # -5 + (-3) = -8
    assert any_int(10, 10, 20) == True  # 10 + 10 = 20
    assert any_int(100, 200, 301) == False  # 100, 200 and 301 doesn't form a valid equation
    assert any_int(1, 2, 4) == False  # 1, 2 and 4 doesn't form a valid equation
    assert any_int(0, -1, -1) == True  # 0 + (-1) = -1
    assert any_int(

# FIND COVERAGE

In [8]:
if flag:
    with open("test_script.py", "w") as file:
        file.write(ans)
else:
    with open("test_script.py", "w") as file:
        file.write(ans[:-3])

In [9]:
!coverage run test_script.py

In [10]:
!coverage report -m

Name             Stmts   Miss  Cover   Missing
----------------------------------------------
test_script.py      33     31     6%   4-8, 14-40
----------------------------------------------
TOTAL               33     31     6%
